# T5rus с промптом + LoRA + псевдотекст

T5rus хорошо знает русский язык но не знает json и плохо умеет генерировать структурированные данные (он на них не учился) поэтому следующий этап - делать промежуточное преобразование в псевдокод и обратно, там мы избежим поломки структуры


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls -la /content/drive/MyDrive/VKR/


In [ ]:
!pip install -U spacy > /dev/null 2>&1
!python -m spacy download ru_core_news_sm > /dev/null 2>&1
!pip install wandb > /dev/null 2>&1
!pip install datasets > /dev/null 2>&1

In [ ]:
import transformers
import datasets
import huggingface_hub
import torch
import wandb

print(transformers.__version__)
print(datasets.__version__)
print(huggingface_hub.__version__)
print(torch.__version__)

Будем напрямую генерировать json по сцене, для этого дообучим T5(Text-To-Text Transfer Transformer) + LoRA

In [ ]:
import json
import torch
import os
import sys
import warnings
import random

import numpy as np

from pathlib import Path
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from transformers import TrainerCallback

from peft import LoraConfig, get_peft_model, TaskType, PeftConfig,PeftModel
from tqdm import tqdm

import matplotlib.pyplot as plt

# отключаем их все чтобы картинку не портили
warnings.filterwarnings("ignore", category=FutureWarning)

DATA_DIR = Path("/content/drive/MyDrive/VKR/dataset/dataset_tmp").expanduser()
#DATA_DIR = Path("/content/drive/MyDrive/VKR/dataset/dataset_small").expanduser()
MODEL_NAME = "sberbank-ai/ruT5-base"

lib_path = os.path.abspath(os.path.join(os.getcwd(), '/content/drive/MyDrive/VKR/'))
sys.path.append(lib_path)

from library.metrics_pseudo import evaluate_all_metrics
from library.safe_compute_metrics import safe_compute_metrics

# перевод в псевдотекст и обратно
from library.utils import json_to_pseudo_text, pseudo_text_to_json

### Промпт с "few shorts" примерами

In [ ]:
# промпт очень большой, поэтому нужно чтобы все влезало
PROMPT = """
Ты должен проанализировать описание сцены и вернуть ответ в специальном псевдоформате.

Твоя задача:
- Найди все объекты, упомянутые в описании, и их признаки.
- Верни результат строго в псевдоформате — одной строкой.

Формат:
объект1 (признак1 признак2) объект2 () объект3 (признак)

Требования:
- Каждый объект указывается один раз.
- Признаки пишутся через пробел внутри круглых скобок.
- Если признаки отсутствуют, используй пустые скобки ().
- Не добавляй объектов или признаков, которых нет в описании.
- В ответе не должно быть никаких пояснений, комментариев или заголовков — только одна строка с результатом.

Примеры:

Описание: Маленький красный стол стоит у окна.
Ответ:
стол (маленький красный) окно ()

Описание: {description}

Ответ:
"""

print(len(PROMPT))

        # Логируем метрики
        # 'f1_objects': только по объектам,
        # 'f1_attributes_macro': Это среднее значение F1 по атрибутам (признакам),
        #                        рассчитанное отдельно для каждого объекта,
        #                        а потом усреднённое по всем объектам.
        #'f1_attributes_weighted': То же, но взвешенное по числу признаков в эталоне
        #'f1_global_obj_attr_pairs': F1 по всем (объект, признак) парам как единому множеству
        #'f1_combined_simple': простое среднее между двух ключевых компонент качества (F1 объекты и F1 признаки)
        #'f1_combined_weighted': взвешенное среднее двух F1-метрик с учетом числа объектов и числа признаков

In [ ]:
def postprocess_text(preds, labels):
    preds_json = []
    labels_json = []

    for pred_str, label_str in zip(preds, labels):
        try:
            pred_json = pseudo_text_to_json(pred_str.strip())
        except Exception as e:
            #print("Ошибка парсинга предсказания:", pred_str, "|", e)
            pred_json = []

        try:
            label_json = pseudo_text_to_json(label_str.strip())
        except Exception as e:
            #print("Ошибка парсинга ground truth:", label_str, "|", e)
            label_json = []

        preds_json.append(pred_json)
        labels_json.append(label_json)

    # посмотрим что выдает
    print(preds_json[0], labels_json[0])

    return preds_json, labels_json

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Если predictions — logits, нужно брать argmax
    if isinstance(predictions, tuple):
        predictions = predictions[0]

    if isinstance(predictions, torch.Tensor):
        predictions = predictions.cpu().numpy()
    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()

    # logits -> ids
    predictions = np.argmax(predictions, axis=-1)

    # Декодируем токены
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Превращаем строки в JSON-списки через внешний postprocess_text
    preds_json, labels_json = postprocess_text(decoded_preds, decoded_labels)

    # Метрики
    f1_objects_list = []
    f1_attributes_macro_list = []
    f1_attributes_weighted_list = []
    f1_combined_simple_list = []
    f1_combined_weighted_list = []
    f1_global_obj_attr_pairs_list = []

    valid = 0

    for pred, label in zip(preds_json, labels_json):

        #print("pred:", pred, type(pred))
        #print("label:", label, type(label))
        #print("-------------------------------")

        if not isinstance(pred, list) or not isinstance(label, list):
            f1_objects_list.append(0.0)
            f1_attributes_macro_list.append(0.0)
            f1_attributes_weighted_list.append(0.0)
            f1_combined_simple_list.append(0.0)
            f1_combined_weighted_list.append(0.0)
            f1_global_obj_attr_pairs_list.append(0.0)
            #print("bad evaluation")
            continue

        try:
            scores = evaluate_all_metrics(label, pred)
            #print("good:",scores)
            f1_objects_list.append(scores["f1_objects"])
            f1_attributes_macro_list.append(scores["f1_attributes_macro"])
            f1_attributes_weighted_list.append(scores["f1_attributes_weighted"])
            f1_combined_simple_list.append(scores["f1_combined_simple"])
            f1_combined_weighted_list.append(scores["f1_combined_weighted"])
            f1_global_obj_attr_pairs_list.append(scores["f1_global_obj_attr_pairs"])
            valid += 1

        except Exception as e:
            f1_objects_list.append(0.0)
            f1_attributes_macro_list.append(0.0)
            f1_attributes_weighted_list.append(0.0)
            f1_combined_simple_list.append(0.0)
            f1_combined_weighted_list.append(0.0)
            f1_global_obj_attr_pairs_list.append(0.0)

    total = len(decoded_preds)

    aggregated_scores = {
        "f1_objects": round(sum(f1_objects_list) / total, 4),
        "f1_attributes_macro": float(round(sum(f1_attributes_macro_list) / total, 4)),
        "f1_attributes_weighted": float(round(sum(f1_attributes_weighted_list) / total, 4)),
        "f1_combined_simple": float(round(sum(f1_combined_simple_list) / total, 4)),
        "f1_combined_weighted": float(round(sum(f1_combined_weighted_list) / total, 4)),
        "f1_global_obj_attr_pairs": float(round(sum(f1_global_obj_attr_pairs_list) / total, 4)),
        "valid_json_rate": float(round(valid / total, 4)),
        "total_samples": float(total),
        "valid_samples": float(valid),
    }
    print("aggregated_scores:", aggregated_scores)
    return aggregated_scores

### Параметры модели и обучения

инъекции будем делать во все слои связанные с вниманием - это должно сделать модель гибче

In [ ]:
lora_rank = 8
lora_alpha = 16
lora_target_modules=["q", "v"]   # в какие слои делаем инъекции
lora_dropout=0.1

per_device_train_batch_size = 8
num_train_epochs = 50

INPUT_SEQ_LENGTH = 1100
OUTPUT_SEQ_LENGTH = 512

# параметры генерации
NUM_BEAMS = 6

# лучше beam подлиннее чем температура, тк у нас структурированный текст
#TEMPERATURE = 0.7

In [ ]:
run = wandb.init(
    entity="shiltsov-da",
    # Set the wandb project where this run will be logged.
    project="vkr-hse-object-detection",
    # Track hyperparameters and run metadata.
    group="T5LoRAtext2textPsC",
    tags=["text2text", "lora", MODEL_NAME],
    config={
        "architecture": "FlanT5-LoRA-text2text-PsC",
        "notebook":"T5ru-LoRA-text2text-parser-v3-Colab.ipynb",
        "base_model": MODEL_NAME,
        "lora_rank": lora_rank,
        "lora_alpha": lora_alpha,
        "lora_target_modules": lora_target_modules,
        "per_device_train_batch_size": per_device_train_batch_size,
        "num_train_epochs": num_train_epochs
    },
)



In [ ]:
# Грузим батчи
def make_target(scene_objects):
    objects_dict = {}
    for obj in scene_objects:
        for name, attrs in obj.items():
            objects_dict[name] = attrs
    ps_text = json_to_pseudo_text([objects_dict])
    return ps_text

data = []
for path in sorted(DATA_DIR.glob("*.jsonl")):
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            item = json.loads(line)
            description = item["description"]
            target = make_target(item["scene"]["objects"])
            data.append({
                "input": PROMPT.format(description=description),
                "target": target,
            })


# Делаем датасет
dataset = Dataset.from_list(data)
dataset = dataset.train_test_split(test_size=0.05, seed=42)
train_ds, val_ds = dataset["train"], dataset["test"]

print(train_ds[0])

In [ ]:
# Токенайзер
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

def preprocess(example):
    inputs = tokenizer(example["input"], padding="max_length", truncation=True, max_length=INPUT_SEQ_LENGTH)
    targets = tokenizer(example["target"], padding="max_length", truncation=True, max_length=OUTPUT_SEQ_LENGTH)
    inputs["labels"] = targets["input_ids"]

    # сохраняем оригинал обратно в пример
    # inputs["target_raw"] = example["target_raw"]
    return inputs

train_ds = train_ds.map(preprocess, batched=False)
val_ds = val_ds.map(preprocess, batched=False)

# Грузим модель + LoRA
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

lora_config = LoraConfig(
    r=lora_rank, # ранг низкоранговой матрицы
    lora_alpha=lora_alpha,
    target_modules=lora_target_modules,
    # target_modules=["q", "k", "v", "o"]
    lora_dropout=lora_dropout,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(model, lora_config)

# Обучение
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/VKR/T5ru_PsC_lora_outputs",
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=4,
    num_train_epochs=num_train_epochs,
    logging_dir="/content/drive/MyDrive/VKR/logs_T5ru_PsC",
    logging_steps=50,
    eval_strategy="epoch",
    eval_accumulation_steps=10, # для маленькой памяти GPU - ск бачей одновременно грузить
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="wandb",
    fp16=True
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# грузим уже обученную
#trainer.train(resume_from_checkpoint=True)

trainer.train()
model.save_pretrained("/content/drive/MyDrive/VKR/T5ru_PsC_lora_outputs")
run.finish()


In [ ]:
run.finish()

### Проверка

In [ ]:
MODEL_DIR = "/content/drive/MyDrive/VKR/T5ru_PsC_lora_outputs"  # путь к fine-tuned модели
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Загрузка модели и токенизатора
print("Loading model...")
config = PeftConfig.from_pretrained(MODEL_DIR)
base_model = T5ForConditionalGeneration.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(base_model, MODEL_DIR)
model = model.to(DEVICE)
model.eval()

tokenizer = T5Tokenizer.from_pretrained(config.base_model_name_or_path)

# Генерация
def predict(description, max_length=OUTPUT_SEQ_LENGTH):
    prompt = PROMPT.format(description=description)
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=INPUT_SEQ_LENGTH
    ).to(DEVICE)

    with torch.no_grad():
        output_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=max_length,
            num_beams=NUM_BEAMS, # попробовать меньше
            #temperature=TEMPERATURE, # параметризовать
            early_stopping=True
        )

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    #print(output_text)
    #print(pseudo_text_to_json(output_text))
    try:
        parsed_json = pseudo_text_to_json(output_text)
    except Exception as e:
        print(f"Ошибка парсинга JSON: {e}")
        print("Сырые данные:", output_text)
        parsed_json = None

    return parsed_json


text = input("Введите описание сцены: ")
result = predict(text)
print("\nПредсказание:\n")
print(json.dumps(result, indent=2, ensure_ascii=False))


In [ ]:
pseudo_text_to_json("кот (черный) кот (черный) кот (черный) дерево (красное) дерево ()")

## Просмотр сколько параметров учили

In [ ]:
def print_trainable_parameters(model):
    trainable_params = 0
    total_params = 0

    for param in model.parameters():
        total_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(f"Всего параметров: {total_params / 1e6:.2f}M")
    print(f"Обучаемых параметров: {trainable_params / 1e6:.2f}M")
    print(f"Доля обучаемых параметров: {100 * trainable_params / total_params:.2f}%")

# Вызов функции после создания модели

model = get_peft_model(model, lora_config)
print_trainable_parameters(model)